Let's make a metric that lets us quantify the uncertainty in a shape measured off a serries of images

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles

In [ ]:
class Shape_precision_metric(metrics.BaseMetric):
    """
    
    Parameters
    ----------
    pixscale : float (0.2)
        The pixelscale of the CCD, arsec/pix (0.2).
    fwhm_object : float (3.)
        The FWHM of the fiducial object who's shape is being measured, arcseconds (3.).
    
        
    stellar_density_limit : float (None)
        XXX--to add, put in a limit where if the stellar density is too high, returns weight of 0.
    
    """
    
    def __init__(self,seeingCol='seeingFwhmGeom', metricName=None, pixscale=0.2, 
                 fwhm_object=3., peak_flux_object=100., stellar_density_limit=None,
                 filters='gri', filterCol='filter'
                 m5Col='fiveSigmaDepth', maps=['StellarDensityMap', 'DustMap'], **kwargs):
        self.seeingCol = seeingCol
        self.m5Col = m5Col
        self.pixscale = pixscale
        self.fwhm_object = fwhm_object
        self.peak_flux_object = peak_flux_object
        self.filterCol = filterCol
        cols =[seeingCol, m5Col, filterCol]
        super().__init__(col=cols, maps=maps, units=units, metricName=metricName, **kwargs)
        
    def run(self, dataSlice, slicepoint=None):
        peak_fluxes = dataSlice[self.seeingCol]*0
        for filtername in self.filters:
            # XXX--apply dust extinction to the peak flux
            # XXX--apply airmass extinction to the peak fluxes
            
        
    